In [1]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'ProsusAI/finbert',
	'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.26.0',
	pytorch_version='1.13.1',
	py_version='py39',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

# predictor.predict({
# 	"inputs": "I like you. I love you",
# })

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
----!

[{'label': 'neutral', 'score': 0.8907915353775024}]

In [12]:
# import predictor
def genSentforpsg(StrText,WordLst):
    # WordLst= ['Exxon Mobil','Exxon']
    outputSentLst=[]
    docs=nlp(StrText)
    for sent in docs.sents:
        # for ent in sent.ents:
        #     if ent.text in WordLst:
        output=predictor.predict({"inputs":sent.text,})
        outputSentLst.append(output[0])
    sentDf= pd.DataFrame(outputSentLst)
    if 'label' in sentDf.columns: 
        sentDict=sentDf.groupby('label')['score'].max().to_dict()
    else:
        sentDict= {'positive':0,}
    return sentDict

ModuleNotFoundError: No module named 'predictor'

In [3]:
# !python -m spacy download en_core_web_sm

In [1]:
import pandas as pd
import numpy as np
import spacy


nlp=spacy.load("en_core_web_sm")
df= pd.read_csv("ford_new_news_links.csv").iloc[:,1:]
dfNew=df[df['Link Content'].isna()==False]

In [ ]:
dfNew['Sentiment']= dfNew.apply(lambda row:genSentforpsg(row['Link Content'],['Ford','ford']),axis=1)

In [ ]:
dfNew['Sentiment'].head(10)

In [ ]:
dfNew[dfNew['Sentiment'].apply(lambda x:len(x))==2]['Sentiment']

In [ ]:
dfNew[dfNew['Sentiment'].apply(lambda x:len(x))==1]['Sentiment']

In [ ]:
dfNew.columns
dfNew[dfNew['Sentiment'].apply(lambda x:len(x))==1]['Title']

In [ ]:
SentimentDF=dfNew['Sentiment'].apply(pd.Series)
dfNewMerge=dfNew.merge(SentimentDF,how='inner',left_index=True,right_index=True)

In [23]:
dfNewMerge.head(10)

,Title,Time,Description,Source,week number,Link Content,Sentiment,negative,neutral,positive
0,3 Stocks to Buy While They Are On Sale,1 week ago,The stock of Ford Motor Company (NYSE: F) appe...,https://finance.yahoo.com/news/3-stocks-buy-wh...,1,Market downturns can be appealing to those who...,"{'negative': 0.9754780530929565, 'neutral': 0....",0.975478,0.945247,0.957985
1,3 High-Yield Dividend Stocks to Buy Hand Over ...,1 week ago,... Ford Motor Company and General Motors mana...,https://finance.yahoo.com/news/3-high-yield-di...,1,Do you know the only thing that gives me pleas...,"{'negative': 0.8808223605155945, 'neutral': 0....",0.880822,0.947828,0.957422
2,Best Dividend Stock 2024: Ford Stock vs. AT&T ...,2 weeks ago,Wall Street Raises The Bar For These Four Stoc...,https://finance.yahoo.com/news/best-dividend-s...,2,Fool.com contributor Parkev Tatevosian compare...,"{'neutral': 0.9495798349380493, 'positive': 0....",NaN,0.949580,0.783337
3,Ford Motor Company (F) Stock Falls Amid Market...,2 weeks ago,Massive News for Rivian Stock Investors. 15h a...,https://finance.yahoo.com/news/ford-motor-comp...,2,The latest trading session saw Ford Motor Comp...,"{'negative': 0.9780202507972717, 'neutral': 0....",0.978020,0.947831,0.955211
4,Ford Motor Company (F) is Attracting Investor ...,2 weeks ago,Over the past month shares of this company hav...,https://finance.yahoo.com/news/ford-motor-comp...,2,Ford Motor Company (F) has been one of the mos...,"{'negative': 0.9243130087852478, 'neutral': 0....",0.924313,0.947831,0.957695
5,Is The Market Rewarding Forward Industries Inc...,2 weeks ago,It is hard to get excited after looking at For...,https://finance.yahoo.com/news/market-rewardin...,2,It is hard to get excited after looking at For...,"{'negative': 0.9695165157318115, 'neutral': 0....",0.969517,0.947645,0.958570
6,GM Ford announce several car models will lose ...,3 weeks ago,... Stock-Split Stock to Buy Before It Does. 2...,https://finance.yahoo.com/news/gm-ford-announc...,3,General Motors and Ford Motor Company announce...,"{'negative': 0.9630406498908997, 'neutral': 0....",0.963041,0.941629,0.849848
7,3 No-Brainer Dividend Stocks for 2024,3 weeks ago,If you're shopping for dividend stocks during ...,https://finance.yahoo.com/news/3-no-brainer-di...,3,F-150 Lightning Flash 1\nSometimes the greates...,"{'negative': 0.7430631518363953, 'neutral': 0....",0.743063,0.950503,0.957894
8,Stock market news today: Stocks close higher a...,3 weeks ago,Stock market news today: Stocks close higher a...,https://finance.yahoo.com/news/stock-market-ne...,3,"US stocks closed higher on Tuesday, with an ei...","{'negative': 0.9758772850036621, 'neutral': 0....",0.975877,0.937800,0.959066
9,13 Most Promising EV Stocks According To Hedge...,3 weeks ago,Some top stocks are Ford Motor Company (NYSE:F...,https://finance.yahoo.com/news/13-most-promisi...,3,"In this piece, we will take a look at the 13 m...","{'negative': 0.970503568649292, 'neutral': 0.9...",0.970504,0.953640,0.951142


In [24]:
dfNewMerge.to_csv("Ford_Sentiment.csv",index=False)